In [ ]:
# Code 1

# 需要pip install selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
from urllib.parse import urlencode, quote_plus
import requests
import time
import re
import os
import json

root_path = os.getcwd()

In [ ]:
# Code 2

# 配置项 1
# --------------------------------------------------------------------------------------------------------------
# 下载模式，可选pdf和mp4
download_mode = "pdf"
# download_mode = "mp4"
# --------------------------------------------------------------------------------------------------------------


# 配置项 2
# --------------------------------------------------------------------------------------------------------------
# 等待页面加载的时间，单位为秒
wait_time = 2

# 如果通过超星学习通登录验证，可以将此项置为False
first_wait = True

# 如果通过学校登录验证，可能在读取课程资源时需要登录，这里填写你输入账号密码所需的时间，单位为秒
login_wait_time = 10
# --------------------------------------------------------------------------------------------------------------


# 配置项 3
# --------------------------------------------------------------------------------------------------------------
# 资源目录，可以在这里填写课程名
file_path = "foo/"

# 进入资源目录
os.chdir(root_path)
os.makedirs(file_path, exist_ok=True)
os.chdir(file_path)
# --------------------------------------------------------------------------------------------------------------


# 配置项 4
# --------------------------------------------------------------------------------------------------------------
# passport，依赖于你的学校

# 超星学习通默认值
passport = "https://passport2.chaoxing.com/"

# 武汉大学
# passport = "http://passport2.mooc.whu.edu.cn/login?fid=1024&refer=http://i.mooc.mooc.whu.edu.cn"
# --------------------------------------------------------------------------------------------------------------


# 配置项 5
# --------------------------------------------------------------------------------------------------------------
# Referer，依赖于你的学校

# 超星学习通默认值
Referer = "https://mooc1.chaoxing.com/ananas/modules/video/index.html?v=2023-1020-1832"

# 武汉大学
# Referer = "http://www.mooc.whu.edu.cn/ananas/modules/video/index.html?v=2023-1027-1832"
# --------------------------------------------------------------------------------------------------------------


# enc 和 cpi，依赖于你的学校，这个值会自动获取
enc = None
cpi = None

# 学习通ServerHost
ServerHost = {
    "moocDomain": "https:" + "//" + "mooc2-ans.chaoxing.com",
    "hostDomain": "https:" + "//" + "mooc2-ans.chaoxing.com",
    "mooc1Domain": "https:" + "//mooc1." + 'chaoxing.com',
    "mobilelearnDomain": "https:" + "//mobilelearn." + 'chaoxing.com',
    "moocHostName": "http://www.fanya." + 'chaoxing.com',
    "purl": "https://p.ananas.chaoxing.com",
    "spaceDomain": "http://i.mooc.chaoxing.com",
    "passporturl": "http://passport2." + 'chaoxing.com',
    "passportlogin": "http://detect." + 'chaoxing.com' + "/api/passport2-onlineinfo.js?key=true&refer=",
    "fanyaurl": "http://course.fanya." + 'chaoxing.com',
    "photoDomain": "http://photo.fanya.chaoxing.com",
    "domainurl_old": "http://fanya.chaoxing.com",
    "yunyadomain": "http://yunya.mooc.chaoxing.com",
    "bjdomain": "http://bj.chaoxing.com",
    "solrDomain": "http://solr.ananas.com:8983",
    "msgDomain": "http://msg.mooc." + 'chaoxing.com',
    "schoolApiDomain": "http://schoolapi.mooc.chaoxing.com",
    "appDomain": "http://apps.ananas.chaoxing.com",
    "csDomain": "http://cs.ananas.chaoxing.com",
    "csDomainHttps2": "https://cs.ananas.chaoxing.com",
    "csDomainHttps": "https://cs-ans.chaoxing.com",
    "cloudDomain": "http://cloud.ananas.chaoxing.com",
    "groupDomain": "https://groupweb.chaoxing.com",
    "uploadDomain": "https://mooc-upload-ans.chaoxing.com",
    "newCloudDomain": "https:" + "//pan-yz.chaoxing.com",
    "xueyinonlineDomain": "https:" + "//www.xueyinonline.com",
    "xueyinonlineChaoXingDomain": "https:" + "//xueyinonline.chaoxing.com",
    "pananas": "http\:\/\/p.ananas\.chaoxing\.com\/star\/origin\/",
    "schoollist": "http://www.fanya." + 'chaoxing.com' + "/school/schoollist.html",
    "moocTJDomain": "https://fystat-ans.chaoxing.com",
    "noteydDomain": "https:" + "//noteyd.chaoxing.com",
}

In [ ]:
# Code 3

# 实现js的to_old_url()
def toOld(courseid, knowledgeId, clazzid):
    mooc1Domain = ServerHost["mooc1Domain"]

    referUrl = mooc1Domain + "/mycourse/studentstudy?chapterId=" + knowledgeId + "&courseId=" + \
        courseid + "&clazzid=" + clazzid + "&cpi=" + cpi + "&enc=" + enc + "&mooc2=1"
    transferUrl = mooc1Domain + "/mycourse/transfer?moocId=" + courseid + \
        "&clazzid=" + clazzid + "&ut=s&refer=" + encodeURIComponent(referUrl)

    referUrl = f"{mooc1Domain}/mycourse/studentstudy?"
    refer_params = {
        'chapterId': knowledgeId,
        'courseId': courseid,
        'clazzid': clazzid,
        'cpi': cpi,
        'enc': enc,
        'mooc2': 1
    }
    referUrl += urlencode(refer_params)

    transferUrl = f"{mooc1Domain}/mycourse/transfer?"
    transfer_params = {
        'moocId': courseid,
        'clazzid': clazzid,
        'ut': 's',
        'refer': referUrl
    }
    transferUrl += urlencode(transfer_params)

    return transferUrl

# 将quote_plus包装成js的同名函数encodeURIComponent()
def encodeURIComponent(string):
    return quote_plus(string)


In [ ]:
# Code 4

# 从所有的iframe中抽取objectid构造XHR_url，并保存到XHR_urls.txt
def process_all_iframes(driver):

    # depth为当前嵌套层数，parent_index为父级iframe的索引
    def recursive_iframe(driver, depth=0, parent_index=0):
        iframes = driver.find_elements(By.TAG_NAME, 'iframe')
        for index, iframe in enumerate(iframes):
            # 切换到当前iframe
            driver.switch_to.frame(iframe)

            # --------------------------------------------------------------------------------------------------------------
            # 工作区: 在这里写每个iframe内要执行的代码

            # 获取当前iframe中的objectid
            objectid = driver.execute_script('''
                if (window && window.frameElement && window.frameElement.getAttribute)
                    return window.frameElement.getAttribute("objectid");
            ''')

            # # Plan B
            # if objectid is None:
            #     # 获取href
            #     href = driver.execute_script('''
            #         element = document.getElementById('downloadUrl');
            #         if(element && element.getAttribute)
            #             return element.getAttribute('href');
            #     ''')

            #     # 抽取objectid
            #     pattern = r"download/([a-f0-9]{32})\?"
            #     match = re.search(pattern, href)
            #     if match:
            #         objectid = match.group(1)

            # 构造XHR_url
            if objectid is not None:

                # 这俩一般意味着没找到
                if re.search(r'\bvoid\b', objectid) or re.search(r'\bjavascript\b', objectid):
                    pass

                else:
                    XHR_url = "https://mooc1.chaoxing.com/ananas/status/" + objectid

                    # 保存到XHR_urls
                    with open("XHR_urls.txt", "a", encoding="utf-8") as f:
                        f.write(XHR_url + "\n")

            # # 打印当前iframe的层级和索引
            # print(
            #     f"Depth: {depth}, Parent index: {parent_index}, Current index: {index}")

            # --------------------------------------------------------------------------------------------------------------

            # 递归调用
            recursive_iframe(driver, depth + 1, index)

            # 切换回父级iframe或主文档
            driver.switch_to.parent_frame()

    # 返回默认页面
    driver.switch_to.default_content()
    recursive_iframe(driver)

In [ ]:
# Code 5

# 选择你使用的浏览器
driver = webdriver.Edge()

# 打开超星，并输入账号密码登录
driver.get(passport)

In [ ]:
# Code 6

# 登录后，进入课程的主页面，然后执行这个代码块

# 切换到最新窗口
all_handles = driver.window_handles
driver.switch_to.window(all_handles[-1])

In [ ]:
# Code 7

# 切换到包含课程内容的iframe，这个代码块通常只能执行一次！
iframe_element = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe_element)

In [ ]:
# Code 8

# 获取课程的所有链接
page_source = driver.page_source

# 保存课程内容页面
with open('page_source.html', 'wt', encoding="utf-8") as f:
    f.write(page_source)

# 从page_source中提取出所有的toOld()调用
pattern = re.compile(r"toOld\('(\d+)', '(\d+)', '(\d+)'\)")
matches = pattern.findall(page_source)

# 从page_source中提取出enc和cpi的值
enc_match = re.search(r'var enc = "([^"]+)"', page_source)
if enc_match:
    enc = enc_match.group(1)

cpi_match = re.search(r'var cpi = (\d+);', page_source)
if cpi_match:
    cpi = cpi_match.group(1)

if enc is None or cpi is None:
    print("enc or cpi not found")
    exit(1)

# 将课程内容的链接保存到course_urls.txt
with open('course_urls.txt', 'wt', encoding='utf-8') as f:
    for match in matches:
        courseid, knowledgeId, clazzid = match
        course_url = toOld(courseid, knowledgeId, clazzid)
        f.write(course_url + '\n')

In [ ]:
# Code 9

# 将course_urls加载到列表
with open('course_urls.txt', 'r', encoding='utf-8') as file:
    course_urls = [line.strip() for line in file.readlines()]

# 去除重复项
course_urls = list(dict.fromkeys(course_urls).keys())

# 保存去重后的course_urls
with open('course_urls.txt', 'wt', encoding='utf-8') as file:
    for url in course_urls:
        file.write(url + '\n')

In [ ]:
# Code 10

# 通过course_urls获取XHR_urls

# 将course_urls加载到列表
with open('course_urls.txt', 'r', encoding='utf-8') as file:
    course_urls = [line.strip() for line in file.readlines()]

# 去除重复项
course_urls = list(dict.fromkeys(course_urls).keys())

# 获取所有XHR_urls
try:
    for i, course_url in enumerate(course_urls):

        # 打开新的标签
        driver.execute_script("window.open('', '_blank');")

        # 切换到新标签
        driver.switch_to.window(driver.window_handles[-1])

        # # 最小化窗口
        # driver.minimize_window()

        # 导航到URL
        driver.get(course_url)

        # 等待页面加载
        time.sleep(wait_time)

        if first_wait:
            first_wait = False
            time.sleep(login_wait_time)

        # --------------------------------------------------------------------------------------------------------------
        # 处理每个iframe
        process_all_iframes(driver)
        # --------------------------------------------------------------------------------------------------------------

        # 关闭当前标签
        driver.close()

        # 切回到原来的标签（如果有需要）
        driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print("Exception:", e)

In [ ]:
# Code 11

# 通过XHR_urls获取resource_urls和resource_names

# 将XHR_urls加载到列表
with open("XHR_urls.txt", "rt", encoding="utf-8") as f:
    XHR_urls = [line.strip() for line in f.readlines()]

# 去除重复项
XHR_urls = list(dict.fromkeys(XHR_urls).keys())

# 创建Session对象
s = requests.Session()
# 在这里设置你的请求头
s.headers.update({
    'Referer': Referer,
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.61"
})

# 保存resource_urls和resource_names、resource_types到文件
try:
    for XHR_url in XHR_urls:
        response = s.get(XHR_url)
        if response.status_code == 200:
            
            rsp = response.json()
            resource_url = rsp.get("pdf")
            resource_type = "pdf"

            if resource_url is None:
                resource_url = rsp.get("http")
                resource_type = "mp4"

            if resource_url is None:
                continue

            with open("resource_urls.txt", "a", encoding="utf-8") as f:
                f.write(resource_url + "\n")

            with open("resource_types.txt", "a", encoding="utf-8") as f:
                f.write(resource_type + "\n")

            resource_name = rsp["filename"]
            with open("resource_names.txt", "a", encoding="utf-8") as f:
                f.write(resource_name + "\n")

except Exception as e:
    print(e)

In [ ]:
# Code 12

# 通过resource_urls下载资源

# 将resource_urls和resource_names、resource_types加载到列表
with open("resource_urls.txt", "rt", encoding="utf-8") as f:
    resource_urls = [line.strip() for line in f.readlines()]
with open("resource_names.txt", "rt", encoding="utf-8") as f:
    resource_names = [line.strip() for line in f.readlines()]
with open("resource_types.txt", "rt", encoding="utf-8") as f:
    resource_types = [line.strip() for line in f.readlines()]

# 去除重复项，这里默认一一对应
resource_urls = list(dict.fromkeys(resource_urls).keys())
resource_names = list(dict.fromkeys(resource_names).keys())

# 下载资源
try:
    for i, resource_url in enumerate(resource_urls):
        
        if resource_types[i] != download_mode:
            continue

        response = s.get(resource_url)
        if response.status_code == 200:
            with open(resource_names[i], "wb") as f:
                f.write(response.content)

        # 等待
        time.sleep(wait_time)

except Exception as e:
    print(e)

In [ ]:
# 切换到最新窗口，调试用
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
# 切换到包含课程内容的iframe，调试用
iframe_element = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe_element)

In [ ]:
# 刷新当前选定的页面，调试用
driver.refresh()

In [ ]:
# 关闭浏览器
driver.quit()